## 功能

在有标签数据集中，正例数目17000左右，负例数目1709个，所以存在严重的不平衡问题，所以我们尝试解决这个问题。

为了解决这个问题，我们需要处理不平衡数据，本py文件使用的是过采样的方法，使用的SMOTE，RandomOverSampler和ADASYN。

特征选择使用的是RFECV（由featureSelectionBasic.ipynb得到）

参考文章：https://beckernick.github.io/oversampling-modeling/

In [44]:
import pymysql
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from collections import Counter
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, precision_recall_fscore_support
from imblearn.over_sampling import SMOTE,ADASYN,RandomOverSampler
import pickle
import json

## 1、获取数据

In [30]:
connection = pymysql.Connect(
    host="localhost",
    port=3306,
    user="root",
    passwd="root",
    charset="utf8",
    db="project_researchers"
)

In [31]:
def getData(connection):
    """
    查询数据，包括特征和标签
    :param connection:
    :return:
    """
    sql_select = """
    SELECT bys_cn, hindex_cn,a_conf+a_journal as a_paper, b_conf + b_journal as b_paper,c_conf + c_journal as c_paper,papernum2017, papernum2016, papernum2015, papernum2014, papernum2013,num_journal,num_conference, project_num, degree, pagerank,degree_centrality,last_year - first_year as diff_year , coauthors_top10000, coauthors_top20000, coauthors_top30000, category, label 
    FROM classifier_isTeacher_xgbc WHERE (label =1 or label = 0) and category is not null
    """
    df = pd.read_sql_query(sql_select, connection)
    all_features = ['bys_cn', 'hindex_cn', 'a_paper', 'b_paper', 'c_paper', 'papernum2017', 'papernum2016', 'papernum2015', 'papernum2014', 'papernum2013', 'num_journal', 'num_conference',  'degree', 'pagerank', 'degree_centrality', 'diff_year', 'coauthors_top10000', 'coauthors_top20000', 'coauthors_top30000', 'category', 'label']
    data = df[all_features]
    return data

data = getData(connection)
print("shape of data:", data.shape)
print("data.info():", data.info())

shape of data: (18694, 21)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18694 entries, 0 to 18693
Data columns (total 21 columns):
bys_cn                18442 non-null float64
hindex_cn             18557 non-null float64
a_paper               18694 non-null int64
b_paper               18694 non-null int64
c_paper               18694 non-null int64
papernum2017          18694 non-null int64
papernum2016          18694 non-null int64
papernum2015          18694 non-null int64
papernum2014          18694 non-null int64
papernum2013          18694 non-null int64
num_journal           18694 non-null int64
num_conference        18694 non-null int64
degree                18623 non-null float64
pagerank              18623 non-null float64
degree_centrality     18623 non-null float64
diff_year             18623 non-null float64
coauthors_top10000    18694 non-null int64
coauthors_top20000    18694 non-null int64
coauthors_top30000    18694 non-null int64
category              18694 non-nul

## 2、处理数据

In [32]:
# 对缺失值进行处理
# Method1：直接将含有缺失字段的值去掉
data = data.dropna()
print("shape of data::", data.shape)
print("data.info()::", data.info())

shape of data:: (18372, 21)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 18372 entries, 0 to 18692
Data columns (total 21 columns):
bys_cn                18372 non-null float64
hindex_cn             18372 non-null float64
a_paper               18372 non-null int64
b_paper               18372 non-null int64
c_paper               18372 non-null int64
papernum2017          18372 non-null int64
papernum2016          18372 non-null int64
papernum2015          18372 non-null int64
papernum2014          18372 non-null int64
papernum2013          18372 non-null int64
num_journal           18372 non-null int64
num_conference        18372 non-null int64
degree                18372 non-null float64
pagerank              18372 non-null float64
degree_centrality     18372 non-null float64
diff_year             18372 non-null float64
coauthors_top10000    18372 non-null int64
coauthors_top20000    18372 non-null int64
coauthors_top30000    18372 non-null int64
category              18372 non-nu

In [33]:
# 将连续值和离散值以及y分开
continuous_features = ['bys_cn', 'hindex_cn', 'a_paper', 'b_paper', 'c_paper', 'papernum2017', 'papernum2016', 'papernum2015', 'papernum2014', 'papernum2013', 'num_journal', 'num_conference',  'degree', 'pagerank', 'degree_centrality', 'diff_year', 'coauthors_top10000', 'coauthors_top20000', 'coauthors_top30000']
discrete_features = ['category']
X_continous = data[continuous_features]
X_discrete = data[discrete_features]
y = data['label']
print("info of X_continuous::", X_continous.info())
print("info of X_discrete::", X_discrete.info())
print("y::", Counter(y))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18372 entries, 0 to 18692
Data columns (total 19 columns):
bys_cn                18372 non-null float64
hindex_cn             18372 non-null float64
a_paper               18372 non-null int64
b_paper               18372 non-null int64
c_paper               18372 non-null int64
papernum2017          18372 non-null int64
papernum2016          18372 non-null int64
papernum2015          18372 non-null int64
papernum2014          18372 non-null int64
papernum2013          18372 non-null int64
num_journal           18372 non-null int64
num_conference        18372 non-null int64
degree                18372 non-null float64
pagerank              18372 non-null float64
degree_centrality     18372 non-null float64
diff_year             18372 non-null float64
coauthors_top10000    18372 non-null int64
coauthors_top20000    18372 non-null int64
coauthors_top30000    18372 non-null int64
dtypes: float64(6), int64(13)
memory usage: 2.8 MB
info of X_c

In [37]:
# 将离散值变成one-hot编码
X_discrete_oneHot = OneHotEncoder(sparse=False).fit_transform(X_discrete)
print(X_discrete_oneHot)

X_all = np.hstack((X_continous, X_discrete_oneHot))
print("shape of X_all::", X_all.shape)

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
shape of X_all:: (18372, 22)


/home/hdj/jupyter/Apps/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## 3、获取需要预测的数据

In [38]:
def getPredictData(connection):
    """
    获取需要预测的数据，包括训练集中的特征
    :param connection:
    :return:
    """
    sql_select = """
    SELECT bys_cn, hindex_cn,a_conf+a_journal as a_paper, b_conf + b_journal as b_paper,c_conf + c_journal as c_paper,papernum2017, papernum2016, papernum2015, papernum2014, papernum2013,num_journal,num_conference, project_num, degree, pagerank,degree_centrality,last_year - first_year as diff_year , coauthors_top10000, coauthors_top20000, coauthors_top30000, category 
    FROM classifier_isTeacher_xgbc WHERE label is null and category is not null
    """
    df = pd.read_sql_query(sql_select, connection)
    all_features = ['bys_cn', 'hindex_cn', 'a_paper', 'b_paper', 'c_paper', 'papernum2017', 'papernum2016', 'papernum2015', 'papernum2014', 'papernum2013', 'num_journal', 'num_conference',  'degree', 'pagerank', 'degree_centrality', 'diff_year', 'coauthors_top10000', 'coauthors_top20000', 'coauthors_top30000', 'category']
    data = df[all_features]
    return data

data_test = getPredictData(connection)
print("shape of data_test:", data_test.shape)
print("data_test.info():", data_test.info())

shape of data_test: (181057, 20)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181057 entries, 0 to 181056
Data columns (total 20 columns):
bys_cn                176811 non-null float64
hindex_cn             180624 non-null float64
a_paper               181057 non-null int64
b_paper               181057 non-null int64
c_paper               181057 non-null int64
papernum2017          181057 non-null int64
papernum2016          181057 non-null int64
papernum2015          181057 non-null int64
papernum2014          181057 non-null int64
papernum2013          181057 non-null int64
num_journal           181057 non-null int64
num_conference        181057 non-null int64
degree                180847 non-null float64
pagerank              180847 non-null float64
degree_centrality     180847 non-null float64
diff_year             180847 non-null float64
coauthors_top10000    181057 non-null int64
coauthors_top20000    181057 non-null int64
coauthors_top30000    181057 non-null int64
category

## 4、处理predictData

In [40]:
# 使用0进行填充
data_test_fill = data_test.fillna(0)
print("info of data_test_fill::", data_test_fill.info())

# 将连续特征和离散特征区分开
X_test_continous = data_test_fill[continuous_features]
X_test_discrete = data_test_fill[discrete_features]

# 离散特征使用one-hot编码
X_test_discrete_oneHot = OneHotEncoder(sparse=False).fit_transform(X_test_discrete)
print(X_test_discrete_oneHot)

X_test_all = np.hstack((X_test_continous, X_test_discrete_oneHot))
print("shape of X_test_all::", X_test_all.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181057 entries, 0 to 181056
Data columns (total 20 columns):
bys_cn                181057 non-null float64
hindex_cn             181057 non-null float64
a_paper               181057 non-null int64
b_paper               181057 non-null int64
c_paper               181057 non-null int64
papernum2017          181057 non-null int64
papernum2016          181057 non-null int64
papernum2015          181057 non-null int64
papernum2014          181057 non-null int64
papernum2013          181057 non-null int64
num_journal           181057 non-null int64
num_conference        181057 non-null int64
degree                181057 non-null float64
pagerank              181057 non-null float64
degree_centrality     181057 non-null float64
diff_year             181057 non-null float64
coauthors_top10000    181057 non-null int64
coauthors_top20000    181057 non-null int64
coauthors_top30000    181057 non-null int64
category              181057 non-null int

/home/hdj/jupyter/Apps/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## 5、使用SMOTE方法

In [46]:
# 通过观察可以发现，当n_features_to_select=15时，f1值可以达到最大值：0.96038，这也是方差分析，rfe和rfecv中最好的效果。
def trainAndTestXGBCrfeSMOTE(X_all, y, X_test_all, n_features_to_select=15):
    
     # RFECV
    estimator = XGBClassifier()
    selector = RFE(estimator=estimator, n_features_to_select = n_features_to_select)
    X_all_rfe = selector.fit_transform(X_all, y) 
    print("Optimal number of features::%d" % selector.n_features_)
    print("Ranking of features:: %s" % selector.ranking_)
    X_test_all_rfe = selector.transform(X_test_all)
    selected_idx = np.where(pd.Series(selector.support_)==True)[0]   # n_features_to_select个选择出来的特征，每一个特征为True
    print("selector.support_::", selector.support_)
    
    # 因为Wrapper离散特征和连续特征需要一起训练搜索特征子集，但是因为离散特征不需要标准化，所以这里需要将其分开
    discrete_idx = list(set([19, 20, 21]) - set(selected_idx))   # 最后3列为离散值
    X_continuous_tmp = pd.DataFrame(X_all_rfe)[list(range(0, len(selected_idx)-len(discrete_idx)))]
    X_discreate_tmp = pd.DataFrame(X_all_rfe)[list(range(len(selected_idx)-len(discrete_idx), len(selected_idx)))]
    X_test_continuous_tmp = pd.DataFrame(X_test_all_rfe)[list(range(0, len(selected_idx)-len(discrete_idx)))]
    X_test_discreate_tmp = pd.DataFrame(X_test_all_rfe)[list(range(len(selected_idx)-len(discrete_idx), len(selected_idx)))]
    
    # 归一化
    ss = StandardScaler()
    X_continuous_new = ss.fit_transform(X_continuous_tmp)
    print("type of X_continuous_new::", type(X_continuous_new))
    print("shape of X_continuous_new::", X_continuous_new.shape)
    X_test_continous_new = ss.transform(X_test_continuous_tmp)
    print("type of X_test_continous_new::", type(X_test_continous_new))
    print("shape of X_test_continous_new::", X_test_continous_new.shape)

    # 将连续值和离散值拼接
    X_all_new = np.hstack((X_continuous_new, X_discreate_tmp))
    print("shape of X_all::", X_all.shape)
    X_test_all_new = np.hstack((X_test_continous_new, X_test_discreate_tmp))
    print("shape of X_test_all::", X_test_all.shape)

    # 划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X_all_new, y, test_size=0.2, random_state=33)
    
    print("shape of X_train::", X_train.shape)
    print("shape of X_test::", X_test.shape)
    print("shape of y_train::", y_train.shape)
    print("Counter of y_train::", Counter(y_train))
    print("shape of y_test::", y_test.shape)
    print("Counter of y_test::", Counter(y_test))
    
    sm = SMOTE(random_state=12, ratio=1.0)
    X_train_res, y_train_res = sm.fit_sample(X_train, y_train)
    print("shape of X_train_res::", X_train_res.shape)
    print("shape of y_train_res::", y_train_res.shape)
    
    
    xgbc = XGBClassifier()
    xgbc.fit(X_train_res, y_train_res)
    y_test_predict = xgbc.predict(X_test)
    print(classification_report(y_test_predict, y_test, target_names=['1', '0'], digits=5))
    
    y_predict = xgbc.predict(X_test_all_new)
    print("y_predict::", Counter(y_predict))

# 调用预测函数
X_all_copy = X_all.copy()
y_copy = y.copy()
X_test_all_copy = X_test_all.copy()
trainAndTestXGBCrfeSMOTE(X_all_copy, y_copy, X_test_all_copy)

Optimal number of features::15
Ranking of features:: [1 1 3 4 1 1 1 2 1 6 1 1 1 1 7 1 5 1 1 1 8 1]
selector.support_:: [ True  True False False  True  True  True False  True False  True  True
  True  True False  True False  True  True  True False  True]
type of X_continuous_new:: <class 'numpy.ndarray'>
shape of X_continuous_new:: (18372, 14)
type of X_test_continous_new:: <class 'numpy.ndarray'>
shape of X_test_continous_new:: (181057, 14)
shape of X_all:: (18372, 22)
shape of X_test_all:: (181057, 22)
shape of X_train:: (14697, 15)
shape of X_test:: (3675, 15)
shape of y_train:: (14697,)
Counter of y_train:: Counter({1: 13387, 0: 1310})
shape of y_test:: (3675,)
Counter of y_test:: Counter({1: 3314, 0: 361})
shape of X_train_res:: (26774, 15)
shape of y_train_res:: (26774,)
              precision    recall  f1-score   support

           1    0.91136   0.66870   0.77140       492
           0    0.95081   0.98995   0.96999      3183

   micro avg    0.94694   0.94694   0.94694      

## 6、使用ADASYN方法

In [48]:
# 通过观察可以发现，当n_features_to_select=15时，f1值可以达到最大值：0.96038，这也是方差分析，rfe和rfecv中最好的效果。
def trainAndTestXGBCrfeADASYN(X_all, y, X_test_all, n_features_to_select=15):
    
     # RFECV
    estimator = XGBClassifier()
    selector = RFE(estimator=estimator, n_features_to_select = n_features_to_select)
    X_all_rfe = selector.fit_transform(X_all, y) 
    print("Optimal number of features::%d" % selector.n_features_)
    print("Ranking of features:: %s" % selector.ranking_)
    X_test_all_rfe = selector.transform(X_test_all)
    selected_idx = np.where(pd.Series(selector.support_)==True)[0]   # n_features_to_select个选择出来的特征，每一个特征为True
    print("selector.support_::", selector.support_)
    
    # 因为Wrapper离散特征和连续特征需要一起训练搜索特征子集，但是因为离散特征不需要标准化，所以这里需要将其分开
    discrete_idx = list(set([19, 20, 21]) - set(selected_idx))   # 最后3列为离散值
    X_continuous_tmp = pd.DataFrame(X_all_rfe)[list(range(0, len(selected_idx)-len(discrete_idx)))]
    X_discreate_tmp = pd.DataFrame(X_all_rfe)[list(range(len(selected_idx)-len(discrete_idx), len(selected_idx)))]
    X_test_continuous_tmp = pd.DataFrame(X_test_all_rfe)[list(range(0, len(selected_idx)-len(discrete_idx)))]
    X_test_discreate_tmp = pd.DataFrame(X_test_all_rfe)[list(range(len(selected_idx)-len(discrete_idx), len(selected_idx)))]
    
    # 归一化
    ss = StandardScaler()
    X_continuous_new = ss.fit_transform(X_continuous_tmp)
    print("type of X_continuous_new::", type(X_continuous_new))
    print("shape of X_continuous_new::", X_continuous_new.shape)
    X_test_continous_new = ss.transform(X_test_continuous_tmp)
    print("type of X_test_continous_new::", type(X_test_continous_new))
    print("shape of X_test_continous_new::", X_test_continous_new.shape)

    # 将连续值和离散值拼接
    X_all_new = np.hstack((X_continuous_new, X_discreate_tmp))
    print("shape of X_all::", X_all.shape)
    X_test_all_new = np.hstack((X_test_continous_new, X_test_discreate_tmp))
    print("shape of X_test_all::", X_test_all.shape)

    # 划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X_all_new, y, test_size=0.2, random_state=33)
    
    print("shape of X_train::", X_train.shape)
    print("shape of X_test::", X_test.shape)
    print("shape of y_train::", y_train.shape)
    print("Counter of y_train::", Counter(y_train))
    print("shape of y_test::", y_test.shape)
    print("Counter of y_test::", Counter(y_test))
    
    sm = ADASYN(random_state=12, ratio=1.0)
    X_train_res, y_train_res = sm.fit_sample(X_train, y_train)
    print("shape of X_train_res::", X_train_res.shape)
    print("shape of y_train_res::", y_train_res.shape)
    
    
    xgbc = XGBClassifier()
    xgbc.fit(X_train_res, y_train_res)
    y_test_predict = xgbc.predict(X_test)
    print(classification_report(y_test_predict, y_test, target_names=['1', '0'], digits=5))
    
    y_predict = xgbc.predict(X_test_all_new)
    print("y_predict::", Counter(y_predict))

# 调用预测函数
X_all_copy = X_all.copy()
y_copy = y.copy()
X_test_all_copy = X_test_all.copy()
trainAndTestXGBCrfeADASYN(X_all_copy, y_copy, X_test_all_copy)

Optimal number of features::15
Ranking of features:: [1 1 3 4 1 1 1 2 1 6 1 1 1 1 7 1 5 1 1 1 8 1]
selector.support_:: [ True  True False False  True  True  True False  True False  True  True
  True  True False  True False  True  True  True False  True]
type of X_continuous_new:: <class 'numpy.ndarray'>
shape of X_continuous_new:: (18372, 14)
type of X_test_continous_new:: <class 'numpy.ndarray'>
shape of X_test_continous_new:: (181057, 14)
shape of X_all:: (18372, 22)
shape of X_test_all:: (181057, 22)
shape of X_train:: (14697, 15)
shape of X_test:: (3675, 15)
shape of y_train:: (14697,)
Counter of y_train:: Counter({1: 13387, 0: 1310})
shape of y_test:: (3675,)
Counter of y_test:: Counter({1: 3314, 0: 361})
shape of X_train_res:: (26692, 15)
shape of y_train_res:: (26692,)
              precision    recall  f1-score   support

           1    0.93352   0.60394   0.73341       558
           0    0.93331   0.99230   0.96190      3117

   micro avg    0.93333   0.93333   0.93333      

## 7、使用随机上采样方法

In [50]:
# 通过观察可以发现，当n_features_to_select=15时，f1值可以达到最大值：0.96038，这也是方差分析，rfe和rfecv中最好的效果。
def trainAndTestXGBCrfeROS(X_all, y, X_test_all, n_features_to_select=15):
    
     # RFECV
    estimator = XGBClassifier()
    selector = RFE(estimator=estimator, n_features_to_select=n_features_to_select)
    X_all_rfe = selector.fit_transform(X_all, y) 
    print("Optimal number of features::%d" % selector.n_features_)
    print("Ranking of features:: %s" % selector.ranking_)
    X_test_all_rfe = selector.transform(X_test_all)
    selected_idx = np.where(pd.Series(selector.support_)==True)[0]   # n_features_to_select个选择出来的特征，每一个特征为True
    print("selector.support_::", selector.support_)
    
    # 因为Wrapper离散特征和连续特征需要一起训练搜索特征子集，但是因为离散特征不需要标准化，所以这里需要将其分开
    discrete_idx = list(set([19, 20, 21]) - set(selected_idx))   # 最后3列为离散值
    X_continuous_tmp = pd.DataFrame(X_all_rfe)[list(range(0, len(selected_idx)-len(discrete_idx)))]
    X_discreate_tmp = pd.DataFrame(X_all_rfe)[list(range(len(selected_idx)-len(discrete_idx), len(selected_idx)))]
    X_test_continuous_tmp = pd.DataFrame(X_test_all_rfe)[list(range(0, len(selected_idx)-len(discrete_idx)))]
    X_test_discreate_tmp = pd.DataFrame(X_test_all_rfe)[list(range(len(selected_idx)-len(discrete_idx), len(selected_idx)))]
    
    # 归一化
    ss = StandardScaler()
    X_continuous_new = ss.fit_transform(X_continuous_tmp)
    print("type of X_continuous_new::", type(X_continuous_new))
    print("shape of X_continuous_new::", X_continuous_new.shape)
    X_test_continous_new = ss.transform(X_test_continuous_tmp)
    print("type of X_test_continous_new::", type(X_test_continous_new))
    print("shape of X_test_continous_new::", X_test_continous_new.shape)

    # 将连续值和离散值拼接
    X_all_new = np.hstack((X_continuous_new, X_discreate_tmp))
    print("shape of X_all::", X_all.shape)
    X_test_all_new = np.hstack((X_test_continous_new, X_test_discreate_tmp))
    print("shape of X_test_all::", X_test_all.shape)

    # 划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X_all_new, y, test_size=0.2, random_state=33)
    
    print("shape of X_train::", X_train.shape)
    print("shape of X_test::", X_test.shape)
    print("shape of y_train::", y_train.shape)
    print("Counter of y_train::", Counter(y_train))
    print("shape of y_test::", y_test.shape)
    print("Counter of y_test::", Counter(y_test))
    
    sm = RandomOverSampler(random_state=12, ratio=1.0)
    X_train_res, y_train_res = sm.fit_sample(X_train, y_train)
    print("shape of X_train_res::", X_train_res.shape)
    print("shape of y_train_res::", y_train_res.shape)
    
    
    xgbc = XGBClassifier()
    xgbc.fit(X_train_res, y_train_res)
    y_test_predict = xgbc.predict(X_test)
    print(classification_report(y_test_predict, y_test, target_names=['1', '0'], digits=5))
    
    y_predict = xgbc.predict(X_test_all_new)
    print("y_predict::", Counter(y_predict))

# 调用预测函数
X_all_copy = X_all.copy()
y_copy = y.copy()
X_test_all_copy = X_test_all.copy()
trainAndTestXGBCrfeROS(X_all_copy, y_copy, X_test_all_copy)

Optimal number of features::15
Ranking of features:: [1 1 3 4 1 1 1 2 1 6 1 1 1 1 7 1 5 1 1 1 8 1]
selector.support_:: [ True  True False False  True  True  True False  True False  True  True
  True  True False  True False  True  True  True False  True]
type of X_continuous_new:: <class 'numpy.ndarray'>
shape of X_continuous_new:: (18372, 14)
type of X_test_continous_new:: <class 'numpy.ndarray'>
shape of X_test_continous_new:: (181057, 14)
shape of X_all:: (18372, 22)
shape of X_test_all:: (181057, 22)
shape of X_train:: (14697, 15)
shape of X_test:: (3675, 15)
shape of y_train:: (14697,)
Counter of y_train:: Counter({1: 13387, 0: 1310})
shape of y_test:: (3675,)
Counter of y_test:: Counter({1: 3314, 0: 361})
shape of X_train_res:: (26774, 15)
shape of y_train_res:: (26774,)
              precision    recall  f1-score   support

           1    0.95845   0.57475   0.71859       602
           0    0.92275   0.99512   0.95757      3073

   micro avg    0.92626   0.92626   0.92626      